In [1]:
%reload_ext typedb_jupyter

In [2]:
%typedb help

Available commands: connect, database, transaction, help
TODO: Print subcommand help


In [3]:
%typedb connect open core 127.0.0.1:1729 admin password

Opened connection to: 127.0.0.1:1729


In [4]:
%typedb database create typedb_jupyter_sample

Created database  typedb_jupyter_sample


In [5]:
%typedb database list

Databases:  typedb_jupyter_graphs, typedb_jupyter_sample


In [6]:
%typedb database delete typedb_jupyter_sample

Deleted database  typedb_jupyter_sample


In [7]:
%typedb database list

Databases:  typedb_jupyter_graphs


In [8]:
%typedb database recreate typedb_jupyter_sample

Recreated database  typedb_jupyter_sample


In [9]:
%typedb database list

Databases:  typedb_jupyter_graphs, typedb_jupyter_sample


In [10]:
# This is not implemented yet: %typedb database schema test_jupyter

In [11]:
%typedb database recreate typedb_jupyter_sample

Recreated database  typedb_jupyter_sample


In [12]:
%typedb transaction open typedb_jupyter_sample schema

Opened schema transaction on database 'typedb_jupyter_sample'  


In [13]:
%%typeql 
define
    attribute name, value string;
    entity person, owns name;


Query completed successfully! (No results to show)


'Stored result in variable: _typeql_result'

In [14]:
%typedb transaction commit

Transaction committed


In [15]:
%typedb transaction open typedb_jupyter_sample write

Opened write transaction on database 'typedb_jupyter_sample'  


In [16]:
%%typeql
insert 
$p isa person, has name "James";

Query returned 1 rows.


p
Entity(person: 0x1e00000000000000000000)


'Stored result in variable: _typeql_result'

In [17]:
%typedb transaction commit

Transaction committed


In [18]:
%typedb transaction open typedb_jupyter_sample read 

Opened read transaction on database 'typedb_jupyter_sample'  


In [19]:
%%typeql 
match $instance isa! $instance-type;

Query returned 2 rows.


instance,instance-type
Entity(person: 0x1e00000000000000000000),EntityType(person)
"Attribute(name: ""James"")",AttributeType(name)


'Stored result in variable: _typeql_result'

In [20]:
print(_typeql_result)
print(_typeql_result[1].get("instance"))

[|  $instance: Entity(person: 0x1e00000000000000000000)  |  $instance-type: EntityType(person)  |, |  $instance: Attribute(name: "James")  |  $instance-type: AttributeType(name)  |]
Attribute(name: "James")


In [21]:
%%typeql 
match $owner isa! $owner_type; entity $owner_type;
fetch {
    "attributes": { $owner.* }
};

Query returned 1 documents.
{
  "attributes": {
    "name": "James"
  }
}


'Stored result in variable: _typeql_result'

In [22]:
%typedb transaction close

Transaction closed
